In [2]:
library(tidyverse)

# Rectangle a nested list into a tidy tibble

**`hoist()`**, **`unnest_longer()`**, and **`unnest_wider()`** provide tools for rectangling, collapsing deeply nested lists into regular columns. **`hoist()`** allows you to selectively pull components of a list-column out in to their own top-level columns, using the same syntax as purrr::pluck(). **`unnest_wider()`** turns each element of a list-column into a column, and **`unnest_longer()`** turns each element of a list-column into a row. **`unnest_auto()`** picks between **`unnest_wider()`** or **`unnest_longer()`** based heuristics described below.

```r
hoist(
  .data,
  .col,
  ...,
  .remove = TRUE,
  .simplify = TRUE,
  .ptype = list(),
  .transform = list()
)

unnest_longer(
  data,
  col,
  values_to = NULL,
  indices_to = NULL,
  indices_include = NULL,
  names_repair = "check_unique",
  simplify = TRUE,
  ptype = list(),
  transform = list()
)

unnest_wider(
  data,
  col,
  names_sep = NULL,
  simplify = TRUE,
  names_repair = "check_unique",
  ptype = list(),
  transform = list()
)

unnest_auto(data, col)
```

# Unnest variants

The three unnest() functions differ in how they change the shape of the output data frame:

* **`unnest_wider()`** preserves the rows, but changes the columns.

* **`unnest_longer()`** preserves the columns, but changes the rows

* **`unnest()`** can change both rows and columns.

# unnest_auto() heuristics

`unnest_auto()` inspects the inner names of the list-col:

* If all elements are unnamed, it uses `unnest_longer()`

* If all elements are named, and there's at least one name in common acros all components, it uses `unnest_wider()`

* Otherwise, it falls back to `unnest_longer(indices_include = TRUE)`.

# Examples

In [3]:
#data, ignore the process of create it

df <- tibble(
  character = c("Toothless", "Dory"),
  metadata = list(
    list(
      species = "dragon",
      color = "black",
      films = c(
        "How to Train Your Dragon",
        "How to Train Your Dragon 2",
        "How to Train Your Dragon: The Hidden World"
       )
    ),
    list(
      species = "blue tang",
      color = "blue",
      films = c("Finding Nemo", "Finding Dory")
    )
  )
)
df

character,metadata
Toothless,"dragon , black , How to Train Your Dragon , How to Train Your Dragon 2 , How to Train Your Dragon: The Hidden World"
Dory,"blue tang , blue , Finding Nemo, Finding Dory"


In [4]:
print(df)

# A tibble: 2 x 2
  character metadata        
  <chr>     <list>          
1 Toothless <named list [3]>
2 Dory      <named list [3]>


In [5]:
# Turn all components of metadata into columns
df %>% unnest_wider(metadata)

character,species,color,films
Toothless,dragon,black,"How to Train Your Dragon , How to Train Your Dragon 2 , How to Train Your Dragon: The Hidden World"
Dory,blue tang,blue,"Finding Nemo, Finding Dory"


In [9]:
# Extract only specified components
df %>% hoist(
    .col = metadata,
    Type = 'species', #column with name Type, value from 'species'
    first_film = list("films", 1L),
    third_film = list("films", 3L)
)

character,Type,first_film,third_film,metadata
Toothless,dragon,How to Train Your Dragon,How to Train Your Dragon: The Hidden World,"black , How to Train Your Dragon 2"
Dory,blue tang,Finding Nemo,NA,"blue , Finding Dory"


In [10]:
df %>% unnest_wider(metadata) %>% unnest_longer(films)

character,species,color,films
Toothless,dragon,black,How to Train Your Dragon
Toothless,dragon,black,How to Train Your Dragon 2
Toothless,dragon,black,How to Train Your Dragon: The Hidden World
Dory,blue tang,blue,Finding Nemo
Dory,blue tang,blue,Finding Dory


In [11]:
# unnest_longer() is useful when each component of the list should form a row
df <- tibble(
  x = 1:3,
  y = list(NULL, 1:3, 4:5)
)

df

x,y
1,NULL
2,"1, 2, 3"
3,"4, 5"


In [12]:
df %>% unnest_longer(y)

x,y
1,NA
2,1
2,2
2,3
3,4
3,5


In [13]:
# Automatically creates names if widening
df %>% unnest_wider(y)

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3
New names:
* `` -> ...1
* `` -> ...2


x,...1,...2,...3
1,NA,NA,NA
2,1,2,3
3,4,5,NA


In [16]:
# But you'll usually want to provide names_sep:
df %>% unnest_wider(y, names_sep = '_')

x,y_1,y_2,y_3
1,NA,NA,NA
2,1,2,3
3,4,5,NA


In [18]:
# And similarly if the vectors are named
df <- tibble(
  x = 1:2,
  y = list(c(a = 1, b = 2), c(a = 10, b = 11, c = 12))
)

df

x,y
1,"1, 2"
2,"10, 11, 12"


In [20]:
df %>% unnest_longer(y)

x,y,y_id
1,1,a
1,2,b
2,10,a
2,11,b
2,12,c


In [21]:
df %>% unnest_wider(y)

x,a,b,c
1,1,2,NA
2,10,11,12
